In [12]:
from datasets import load_dataset
import matplotlib.pyplot as plt
from transformers import EfficientNetModel
from torchvision.transforms.functional import pil_to_tensor
from datasets import DatasetDict
import torch
import numpy as np
from torchvision.transforms import Resize, Compose, Normalize

In [13]:
ds = load_dataset('i4ds/radio-sunburst-ecallisto')

In [14]:
subset_size = 100

# Selecting the first 100 examples from each split
train_subset = ds['train'].select(range(subset_size))
test_subset = ds['test'].select(range(subset_size))
val_subset = ds['validation'].select(range(subset_size))

dd = DatasetDict()
dd['train'] = train_subset
dd['test'] = test_subset
dd['validation'] = val_subset

In [15]:
def to_torch_tensor(example):
    import numpy as np
    from torchvision.transforms.functional import pil_to_tensor
    # Convert the PIL image to a NumPy array for checking
    np_image = np.array(example['image'])

    # Check if any value is above 255
    if np.any(np_image > 255):
        print("Found value above 255 in example:", example)

    # Check if the image is not uint8
    if np_image.dtype != np.uint8:
        print("Image is not of type uint8 in example:", example)

    # Convert the PIL image to a tensor
    example['image'] = pil_to_tensor(example['image'])
    return example


def scale(example, max_value=255):
    print(example['image'])
    example['image'] = torch.div(example['image'], max_value)
    return example

In [16]:
dd = dd.map(to_torch_tensor, num_proc=8)

Map (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

In [1]:
dd

NameError: name 'dd' is not defined

In [ ]:
mean = torch.mean(torch.stack(dd['train']['image']))
std = torch.std(torch.stack(dd['train']['image']))

KeyError: "Column train not in the dataset. Current columns in the dataset: ['image', 'label', 'antenna', 'datetime']"

In [ ]:
normalize = Normalize(mean=mean, std=std)
size = (224, 244)

In [ ]:
_transforms = Compose([Resize(size), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
food = food.with_transform(transforms)

In [ ]:
model = EfficientNetModel.from_pretrained('google/efficientnet-b0')